In [ ]:
# Install required libraries
!pip install transformers torch scikit-learn

In [ ]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader
from torch.nn import CrossEntropyLoss
from torch.optim import AdamW
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from transformers import MobileBertTokenizer, MobileBertForSequenceClassification

In [ ]:
# Load your SMS dataset
df = pd.read_csv('/content/sms_spam.csv')  # change to your dataset path

# Label encoding
df['label'] = df['label'].map({'ham': 0, 'spam': 1})

# Split
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)

In [ ]:
# Tokenizer + Model
tokenizer = MobileBertTokenizer.from_pretrained("google/mobilebert-uncased")
model = MobileBertForSequenceClassification.from_pretrained("google/mobilebert-uncased", num_labels=2)

# Class weights
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
weights = torch.tensor(class_weights, dtype=torch.float).to(model.device)
loss_fn = CrossEntropyLoss(weight=weights)

In [ ]:
# Tokenize
train_encodings = tokenizer(X_train.tolist(), truncation=True, padding=True)
train_dataset = TensorDataset(
    torch.tensor(train_encodings['input_ids']),
    torch.tensor(train_encodings['attention_mask']),
    torch.tensor(y_train.values)
)
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)

# Optimizer
optimizer = AdamW(model.parameters(), lr=2e-5)

In [ ]:
# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.train()
for epoch in range(2):
    for batch in train_dataloader:
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        loss = loss_fn(logits, labels)
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1} Loss: {loss.item()}")

In [ ]:
# Example SMS prediction
def predict_sms(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)
        pred = torch.argmax(outputs.logits, dim=1).item()
    return "SPAM" if pred == 1 else "HAM"

# Example test
print(predict_sms("Congratulations! You've won a free ticket. Reply now!"))
print(predict_sms("Hey, are you coming to class tomorrow?"))

In [ ]:
# Save model & tokenizer
model.save_pretrained("./fine_tuned_mobilebert_sms/")
tokenizer.save_pretrained("./fine_tuned_mobilebert_sms/")
!zip -r mobilebert_sms_model.zip fine_tuned_mobilebert_sms